In [7]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = '../model/bottleneck_fc_model.h5'
# train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
# validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
# nb_train_samples = 1462
# nb_validation_samples = 167
train_data_dir = '../data/Pandora18K_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
nb_train_samples = 14313
nb_validation_samples = 1772

epochs = 50
batch_size = 16


In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, (nb_train_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_train.npy', 'w'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, (nb_validation_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_validation.npy', 'w'),
        bottleneck_features_validation)

Found 14313 images belonging to 18 classes.
Found 1772 images belonging to 18 classes.


In [9]:
from keras.utils.np_utils import to_categorical

train_data = np.load(open('../model/bottleneck_features_train.npy'))
#full
train_labels = np.array([0]*684+[1]*848+[2]*851+[3]*577+[4]*1021+[5]*862+[6]*659+[7]*813+[8]*880+[9]*1008+[10]*791+[11]*937+[12]*701+[13]*576+[14]*967+[15]*650+[16]*836+[17]*652)

#small
#train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)
train_labels = to_categorical(train_labels, num_classes=18)

validation_data = np.load(open('../model/bottleneck_features_validation.npy'))
#full
validation_labels = np.array([0]*72+[1]*102+[2]*108+[3]*66+[4]*108+[5]*100+[6]*91+[7]*90+[8]*117+[9]*130+[10]*95+[11]*134+[12]*104+[13]*64+[14]*107+[15]*87+[16]*122+[17]*75)

#small
#validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
validation_labels = to_categorical(validation_labels, num_classes=18)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dropout(0))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(18, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path,overwrite=True)

Train on 14313 samples, validate on 1772 samples
Epoch 1/50
14313/14313 [==============================] - 10s - loss: 2.6069 - acc: 0.1898 - val_loss: 2.2142 - val_acc: 0.2585
Epoch 2/50
14313/14313 [==============================] - 10s - loss: 2.2905 - acc: 0.2454 - val_loss: 2.1480 - val_acc: 0.2675
Epoch 3/50
14313/14313 [==============================] - 10s - loss: 2.2487 - acc: 0.2623 - val_loss: 2.1665 - val_acc: 0.2573
Epoch 4/50
14313/14313 [==============================] - 10s - loss: 2.2402 - acc: 0.2724 - val_loss: 2.2083 - val_acc: 0.2760
Epoch 5/50
14313/14313 [==============================] - 10s - loss: 2.2232 - acc: 0.2874 - val_loss: 2.2428 - val_acc: 0.2630
Epoch 6/50
14313/14313 [==============================] - 10s - loss: 2.2001 - acc: 0.2916 - val_loss: 2.3271 - val_acc: 0.2647
Epoch 7/50
14313/14313 [==============================] - 10s - loss: 2.2017 - acc: 0.2924 - val_loss: 2.5083 - val_acc: 0.2500
Epoch 8/50
14313/14313 [==============================]

KeyboardInterrupt: 

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional


best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials())
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)